## Permutation

Permutation group element for integer $n$

In [1]:
from __future__ import annotations
from itertools import permutations

In [3]:
per_test= Permutation(5, [2,5,4,3,1])

In [4]:
per_inverse = per_test.inverse()
print(per_test.plist)
print(per_inverse.plist)

[2, 5, 4, 3, 1]
[5, 1, 4, 3, 2]


In [5]:
(per_inverse * per_test).plist

[1, 2, 3, 4, 5]

## permutation notation
Cauchy: 2 line notation

Cyclic, Canonical Cyclic

In [6]:
print(per_test.notation_cauchy())

[1, 2, 3, 4, 5]
[2, 5, 4, 3, 1]


In [7]:
print(per_test.notation_cyclic())
print(per_inverse.notation_cyclic())

[[1, 2, 5], [3, 4]]
[[1, 5, 2], [3, 4]]


In [9]:
print(per_test.notation_cyclic(canonical=True, string= True))
print(per_inverse.notation_cyclic(canonical=True, string=True))

(43)(512)
(43)(521)


In [2]:
class Permutation:
    @classmethod
    def get_permutations(cls, n, per=False): # get list of permutations for given 'n'
        per = permutations(range(1, n+1), n)
        if per:
            return [cls(n, p) for p in per]
        else:
            return [ p for p in per]
    @classmethod
    def reverse_permutation(cls, n):
        return cls(n, range(n, 0, -1)) 


    def __init__(self, n, plist):
        if len(plist) != n:
            raise ValueError(f"{n} must be same with len(plist) = {len(plist)}")

        if sum(plist) != int(n* (n+1)/2):
            raise ValueError(f"plist does not satisfy permutation proeprty.\n All [0, n-1] values must be in plist. \n {plist}")

        self.n = n
        self.plist = plist
    def __getitem__(self, key):
        if type(key) != int:
            raise ValueError(f"key must be an integer type element: {key}")
        if key < 1 or key > self.n:
            raise IndexError(f"{key} must be in [1, {self.n}] range.")
        
        return self.plist[key-1]
    def __mul__(self, other: Permutation) -> Permutation:
        if self.n != other.n:
            raise ValueError(f"{self.n} and {other.n} are not same.")
        
        rlist = [other[x] for x in self.plist]
        return Permutation(self.n, rlist)

    def index_mul(self, other, oper=False):

        rlist = [self[x] for x in other.plist]
        if oper:
            self.plist= rlist
        else:
            return Permutation(self.n, rlist)

    def index_mul_partial(self, sub_permutation, oper = False): #Work on indexing
        if not isinstance( sub_permutation, Permutation):
            raise ValueError(f"Given parameter must be \'Permutation\' object. \n Current object:{type(sub_permutation)}")
        if self.n %sub_permutation.n != 0:
            raise ValueError(f"Sub permutation must have a divisor of main permuatain size as its size\n main:{self.n}, sub:{sub_permutation.n}")

        n = int(self.n / sub_permutation.n)
        m = sub_permutation.n
        
        rlist = []
        for i in range(0,n):
            tem_rlist = [self.plist[x+m*i -1] for x in sub_permutation.plist]
            rlist = rlist + tem_rlist
        
        if oper:
            self.plist = rlist
        else:
            return Permutation(self.n, rlist)

    def permute_to_list_index(self, li):
        if len(li) != self.n:
            raise ValueError(f"{len(li)} ! = {self.n}")
        
        rlist = [li[x-1] for x in self.plist ]
        return rlist

    def inverse(self)-> Permutation:
        ilist= [self.plist.index(x)+1 for x in range(1, self.n+1)]
        return Permutation(self.n, ilist)
    
    def notation_cauchy(self):
        return f"{list(range(1, self.n+1))}\n{self.plist}"

    
    def __canocial_order(self, cyclist):
        c_clist_1 = []
        c_clist_2 = []
        for li in cyclist:
            n = len(li)
            nmax = li.index(max(li))
            c_clist_1.append([li[x] for x in range(nmax-n, nmax)])
        maxindex = [x[0] for x in c_clist_1]
        s_index = sorted(maxindex)

        for i in s_index:
            c_clist_2.append(c_clist_1[maxindex.index(i)])
        
        return c_clist_2

    def notation_cyclic(self, canonical=False, string= False, sep=''):
        cycliclist = []
        index = list(range(1, self.n+1))
        i =1

        while True:
            ilist = []
            ilist.append(i)
            index.remove(i)
            
            sig_i = self[i]
            while sig_i in index:
                ilist.append(sig_i)
                index.remove(sig_i)
                sig_i = self[sig_i]
            
            cycliclist.append(ilist)

            if len(index) > 0:
                i = min(index)
            else: 
                break
        if canonical:
            cycliclist = self.__canocial_order(cycliclist)
        
        if string:
            cycstr = ''
            for li in cycliclist:
                cycstr = cycstr+ '('+f'{sep}'.join(map(str,li))+')'
            return cycstr
        else:
            return cycliclist



In [18]:
testlist = [i for i in range(1,8+1)]

In [19]:
testlist

[1, 2, 3, 4, 5, 6, 7, 8]

In [66]:
def split_list(li, n):
    if len(li) %n !=0:
        raise ValueError(f"The length of sublist, {n}, must be a divider of original list, {len(li)}. ")
    
    rlist =[]
    for i in range(0, int(len(li)/n)):
        ni = n*i
        rlist.append([li[ni: ni+n]][0])

    return rlist
def sig_rearrange(nn, ns, split=False): #1
        n_l = nn*ns

        nlist = [i+1 for i in range(0, n_l)]
        nlist_splited = split_list(nlist, int(ns/2))
        rlist=[]

        if split:
            for i in range(0, nn):
                rlist.append(nlist_splited[i]+nlist_splited[2*nn-i-1])
        else:
            for i in range(0,nn):
                rlist = rlist + nlist_splited[i] + nlist_splited[2*nn-i-1]

        return rlist

In [44]:
print(testlist[0:2])
print(testlist[2:4])
print(testlist[4:6])

[1, 2]
[3, 4]
[5, 6]


In [72]:
rlist = []
for arr in sig_rearrange(8, 4, split=True):
    for i in per_4:
        rlist.append(arr[i-1])


In [73]:
rlist

[32,
 1,
 2,
 31,
 30,
 3,
 4,
 29,
 28,
 5,
 6,
 27,
 26,
 7,
 8,
 25,
 24,
 9,
 10,
 23,
 22,
 11,
 12,
 21,
 20,
 13,
 14,
 19,
 18,
 15,
 16,
 17]

In [53]:
nn=2
ns = 4
n_l = nn*ns

nlist = [i+1 for i in range(0, n_l)]
nlist_splited = split_list(nlist, int(ns/2))
        

In [54]:
nlist_splited

[[1, 2], [3, 4], [5, 6], [7, 8]]